### _Activity feature extractor: Network structures_
Author : @leopauly | cnlp@leeds.ac.uk <br>
Description : Playing around different custom network structures for extracting action features from a video <br>
Status : Rough work

#### Suggested network structures

In [17]:
## Defining placeholders in tf
data_1 = tf.placeholder(tf.float32, [None, cluster_length, height, width, channel]) #step_size=No: of frames in video sequence
data_2 = tf.placeholder(tf.float32, [None, cluster_length, height, width, channel])
data_3 = tf.placeholder(tf.float32, [None, cluster_length, height, width, channel])
target = tf.placeholder(tf.float32, [None, nb_classes])

In [19]:
## Suggested Network structure 2a: Conv3D followed by LSTM

def cnn_modeling():
    input_cnn = Input(shape=(cluster_lengh, height, width, channel))
    
    cnn_1=Conv3D(filters=64, kernel_size=(3,3,3),padding='same',activation='relu')(input_cnn)
    pool_1=MaxPooling3D(pool_size=(1, 2, 2),strides=(1, 2, 2),padding='valid')(cnn_1)
    
    cnn_2=Conv3D(filters=128, kernel_size=(3,3,3),padding='same',activation='relu')(pool_1)
    pool_2=MaxPooling3D(pool_size=(2, 2, 2),strides=(2, 2, 2),padding='valid')(cnn_2)
    
    cnn_3a=Conv3D(filters=256, kernel_size=(3,3,3),padding='same',activation='relu')(pool_2)
    cnn_3b=Conv3D(filters=256, kernel_size=(3,3,3),padding='same',activation='relu')(cnn_3a)
    pool_3=MaxPooling3D(pool_size=(2,2,2),strides=(2,2,2),padding='valid')(cnn_3b)
    
    #cnn_4a=Conv3D(filters=512, kernel_size=(3,3,3),padding='same',activation='relu')(pool_3)
    #cnn_4b=Conv3D(filters=512, kernel_size=(3,3,3),padding='same',activation='relu')(cnn_4a)
    #pool_4=MaxPooling3D(pool_size=(2, 2, 2),strides=(2, 2, 2),padding='valid')(cnn_4b)
    
    #cnn_5a=Conv3D(filters=64, kernel_size=(3,3,0),padding='same',activation='relu')(pool_4)
    #cnn_5b=Conv3D(filters=64, kernel_size=(3,3,0),padding='same',activation='relu')(cnn_5a)
    #ßpool_5=MaxPooling3D(pool_size=(time_step, 8, 8),strides=(2, 4, 4),padding='valid')(cnn_5b)
    
    #flat= Flatten()(pool_3)
    flat=Reshape([1,-1])(pool_3)
    
    # FC layers group
    fc_1=Dense(4096, activation='relu')(flat)
    drop_1=Dropout(.5, name='do1')(fc_1)
    fc_2=Dense(4096, activation='relu')(drop_1)
    drop_2=Dropout(.5, name='do2')(fc_2)
    fc_2=Dense(487, name='fc8')(drop_2)
    
    model_cnn=Model(input_cnn,fc_2)
    print(model_cnn.summary())
    return model_cnn

input_1 = Input(shape=(cluster_length, height, width, channel))
input_2 = Input(shape=(cluster_length, height, width, channel))
input_3 = Input(shape=(cluster_length, height, width, channel))

model_cnn=cnn_modeling()
layer1_1=model_cnn(inputs=input_1)
layer1_2=model_cnn(inputs=input_2)
layer1_3=model_cnn(inputs=input_3)

concat= keras.layers.concatenate([layer1_1, layer1_2,layer1_3],1)
layer2=keras.layers.LSTM(lstm_h_units,return_sequences=True)(concat)
layer3= Reshape([3*lstm_h_units])(layer2)
model=Model([input_1,input_2,input_3],layer2)
logit_out=model([data_1,data_2,data_3])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 5, 150, 300, 1)    0         
_________________________________________________________________
conv3d_35 (Conv3D)           (None, 5, 150, 300, 64)   1792      
_________________________________________________________________
max_pooling3d_26 (MaxPooling (None, 5, 75, 150, 64)    0         
_________________________________________________________________
conv3d_36 (Conv3D)           (None, 5, 75, 150, 128)   221312    
_________________________________________________________________
max_pooling3d_27 (MaxPooling (None, 2, 37, 75, 128)    0         
_________________________________________________________________
conv3d_37 (Conv3D)           (None, 2, 37, 75, 256)    884992    
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 2, 37, 75, 256)    1769728   
__________

In [9]:
## Suggested Network structure 2b: Conv3D followed by LSTM
input_1 = Input(shape=(time_step, height, width, channel))
input_2 = Input(shape=(time_step, height, width, channel))
input_3 = Input(shape=(time_step, height, width, channel))

conv3D=Conv3D(filters=1, kernel_size=(3,3,3),padding='same')
layer1_1=conv3D(inputs=input_1)
layer1_2=conv3D(inputs=input_2)
layer1_3=conv3D(inputs=input_3)

max_pool=MaxPooling3D(pool_size=(time_step, 8, 8),strides=(2, 4, 4),padding='valid')
layer_2a = max_pool(layer1_1)
layer_2b = max_pool(layer1_2)
layer_2c = max_pool(layer1_3)

layer_3a = Reshape([1,-1])(layer_2a)
layer_3b = Reshape([1,-1])(layer_2b)
layer_3c = Reshape([1,-1])(layer_2c)

#layer_3a = Flatten()(layer_2a)
#layer_3b = Flatten()(layer_2b)
#layer_3c = Flatten()(layer_2c)

concat= keras.layers.concatenate([layer_3a, layer_3b,layer_3c],1)
layer4=keras.layers.LSTM(lstm_h_units,return_sequences=True)(concat)
layer5= Reshape([lstm_h_units*3])(layer4)
out = Dense(nb_classes)(layer5)

model=Model([input_1,input_2,input_3],out)
logit_out=model([data_1,data_2,data_3])
shape=tf.shape(logit_out)
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_21 (InputLayer)            (None, 50, 150, 300,  0                                            
____________________________________________________________________________________________________
input_22 (InputLayer)            (None, 50, 150, 300,  0                                            
____________________________________________________________________________________________________
input_23 (InputLayer)            (None, 50, 150, 300,  0                                            
____________________________________________________________________________________________________
conv3d_33 (Conv3D)               (None, 50, 150, 300,  28          input_21[0][0]                   
                                                                   input_22[0][0]          

In [4]:
## Suggested Network structure 1: Conv3D followed by ConvLSTM
input_1 = Input(shape=(3, height, width, channel))
input_2 = Input(shape=(3, height, width, channel))
input_3 = Input(shape=(3, height, width, channel))

conv3D=Conv3D(filters=1, kernel_size=(3,3,0),padding='same')
layer1_1=conv3D(inputs=input_1)
layer1_2=conv3D(inputs=input_2)
layer1_3=conv3D(inputs=input_3)

concatenated = keras.layers.concatenate([layer1_1, layer1_2,layer1_3],1)
model_out=ConvLSTM2D(filters=1, kernel_size=(5, 5),padding='SAME', return_sequences=True)(concatenated)

model=Model([input_1,input_2,input_3],model_out)
inter_out=model([data_1,data_2,data_3])
print(model.summary())

layer2=tf.nn.max_pool3d(inter_out,ksize=(1,1,4,4,1),strides=(1,1,4,4,1),padding='VALID')
size1=tf.shape(layer2)
shape1=tf.shape(layer2)
fc = tf.reshape(layer2,[batch_size, 133200]) #shape1[1]*shape1[2]*shape1[3]*shape1[4]])  #138750
#shape2=tf.shape(fc)
out=tf.layers.dense(fc,nb_classes)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 3, 150, 300, 1 0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 3, 150, 300, 1 0                                            
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 3, 150, 300, 1 0                                            
____________________________________________________________________________________________________
conv3d_1 (Conv3D)                (None, 3, 150, 300, 1 1           input_1[0][0]                    
                                                                   input_2[0][0]           

In [5]:
predictions= tf.nn.softmax(logit_out)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=target,logits=logit_out))
optimizer = tf.train.AdamOptimizer(lr_rate)
minimize = optimizer.minimize(cross_entropy)
mistakes = tf.not_equal(tf.argmax(target, 1), tf.argmax(predictions, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))

## Defining variables for writing summary
#tf.summary.histogram("predictions",predictions )
tf.summary.scalar("cross_entropy",cross_entropy )
tf.summary.scalar("test_error",error )
summary_op = tf.summary.merge_all()